In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from sklearn.metrics import jaccard_score

plt.rcParams["figure.figsize"] = (10,10)

sns.set()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# reading inputs

politifact = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/politifact_categories.csv")
gossipcop = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/gossipcop_categories.csv")
kaggle = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/kaggle_kdd_features_categories.csv")
liar = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/liar_features_categories.csv")
coaid = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/coaid_features_categories.csv")
celebrity = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/celebrity_categories.csv")
fakenews = pd.read_csv("/content/drive/MyDrive/paper-projects/hate-fake-online-asonam/fakenews_categories.csv")

In [4]:
#updating kaggle labels
dict_replace = {"1": "fake", "0":"real"}
kaggle["label"] = kaggle["label"].map(dict_replace)

#updating liar labels
dict_replace = {0: "fake", 3:"real"}
liar["label"] = liar["label"].map(dict_replace)

#updating coaid labels
dict_replace = {"Fake": "fake", "Real":"real"}
coaid["label"] = coaid["label"].map(dict_replace)

## Top categories for each attributes

In [ ]:
# coaid[:100].pivot_table(index=['news_category'],columns='toxicity',aggfunc='mean').plot(kind='bar')
feature_list = ['toxicity', 'severe_toxicity', 'insult', 'profanity',
       'threat', 'inflammatory', 'identity_attack']

def print_top_means(df, feature_list, topk=5):
  for f in feature_list:
    print(f)
    cats = df["news_category"].unique()
    results_list = []
    for c in cats:
      aux = df[df["news_category"]==c]
      try:
        results_list.append((c,aux[f].mean()))
      except Exception as e:
        print(e)  

    results_list.sort(key=lambda tup: tup[1], reverse=True)
    print(results_list[:topk])

print_top_means(liar[liar["label"]=="fake"], feature_list)
print("DIVISAO")
print_top_means(liar[liar["label"]=="real"], feature_list)

toxicity
[('WORLDPOST', 0.42631236), ('WEDDINGS', 0.26858782000000003), ('QUEER VOICES', 0.198771960842), ('CRIME', 0.19165191454545452), ('COMEDY', 0.17168192000000002)]
severe_toxicity
[('WORLDPOST', 0.0682127), ('QUEER VOICES', 0.026053652193212335), ('CRIME', 0.0165463415), ('BLACK VOICES', 0.016276910675), ('ENTERTAINMENT', 0.014432842833333334)]
insult
[('WORLDPOST', 0.3562648), ('COMEDY', 0.21867940600000002), ('QUEER VOICES', 0.13665264594100002), ('CRIME', 0.12467803272727274), ('TECH', 0.11755871400000001)]
profanity
[('WEDDINGS', 0.179518305), ('WORLDPOST', 0.16599734), ('QUEER VOICES', 0.08133006823643335), ('WOMEN', 0.054730983428571425), ('MEDIA', 0.05225027651999999)]
threat
[('CRIME', 0.22079468054545456), ('BLACK VOICES', 0.148336939), ('DIVORCE', 0.09004273725), ('STYLE & BEAUTY', 0.07657916680000001), ('WOMEN', 0.07094749578571428)]
inflammatory
[('LATINO VOICES', 0.65876186), ('WEIRD NEWS', 0.55178344), ('CRIME', 0.5381386081818182), ('RELIGION', 0.48196106235294117

In [ ]:
print_top_means(kaggle[kaggle["label"]=="real"], feature_list)
print("DIVISAO")
print_top_means(kaggle[kaggle["label"]=="fake"], feature_list)

toxicity
[('GREEN', 0.25219622), ('COMEDY', 0.2093048989222222), ('CRIME', 0.17645045109836063), ('WOMEN', 0.16862934807692304), ('BLACK VOICES', 0.14747849795918366)]
severe_toxicity
[('COMEDY', 0.02488550076111111), ('PARENTING', 0.022140611158064517), ('GREEN', 0.01712155), ('WELLNESS', 0.016299079589600002), ('WOMEN', 0.013396775289230771)]
insult
[('COMEDY', 0.19308764238888892), ('GREEN', 0.15282036), ('CRIME', 0.1472049575737705), ('MEDIA', 0.126708843375), ('SCIENCE', 0.12160235625)]
profanity
[('GREEN', 0.13982697), ('ARTS & CULTURE', 0.09677053), ('COMEDY', 0.09151198732222222), ('PARENTING', 0.07869529892580644), ('WOMEN', 0.07637421356153845)]
threat
[('GREEN', 0.2848881), ('CRIME', 0.07658085672131146), ('BLACK VOICES', 0.041796760785714274), ('COMEDY', 0.037155130388888895), ('ARTS & CULTURE', 0.035133462)]
inflammatory
[('WEIRD NEWS', 0.3196242870869565), ('GREEN', 0.29989758), ('STYLE', 0.28435987), ('COLLEGE', 0.28002903), ('PARENTING', 0.2641543806451613)]
identity_at

In [ ]:
print_top_means(coaid[coaid["label"]=="real"], feature_list)
print("DIVISAO")
print_top_means(coaid[coaid["label"]=="fake"], feature_list)

toxicity
[('WOMEN', 0.06294653690909092), ('WEIRD NEWS', 0.060527626), ('ARTS & CULTURE', 0.05852979), ('MONEY', 0.044405237), ('DIVORCE', 0.04131502)]
severe_toxicity
[('WEIRD NEWS', 0.0035818522), ('WOMEN', 0.003062679427272727), ('ARTS & CULTURE', 0.0026455172), ('DIVORCE', 0.0021550562), ('MONEY', 0.002110469)]
insult
[('WOMEN', 0.058098539818181826), ('WEIRD NEWS', 0.049209762), ('ARTS & CULTURE', 0.046628658), ('PARENTS', 0.0373328891), ('BLACK VOICES', 0.029986039)]
profanity
[('ARTS & CULTURE', 0.032929316), ('WOMEN', 0.03115733801818182), ('WEIRD NEWS', 0.023999332), ('BLACK VOICES', 0.015522823500000001), ('PARENTS', 0.0143158803)]
threat
[('WOMEN', 0.038821840272727275), ('POLITICS', 0.01841112997008055), ('PARENTS', 0.017506716999999998), ('DIVORCE', 0.017501174), ('ARTS & CULTURE', 0.016835894)]
inflammatory
[('WEIRD NEWS', 0.45617947), ('WEDDINGS', 0.27245054), ('CRIME', 0.24157225999999998), ('WOMEN', 0.19525055718181816), ('ARTS & CULTURE', 0.1843044)]
identity_attack
[

In [ ]:
print_top_means(politifact[politifact["label"]=="real"], feature_list)
print("DIVISAO")
print_top_means(politifact[politifact["label"]=="fake"], feature_list)

toxicity
[('BLACK VOICES', 0.16965903), ('QUEER VOICES', 0.123736724), ('COMEDY', 0.11200782800000002), ('THE WORLDPOST', 0.10580590283333334), ('POLITICS', 0.09072671962514794)]
severe_toxicity
[('QUEER VOICES', 0.007847377), ('BLACK VOICES', 0.006866455), ('THE WORLDPOST', 0.005201859949999999), ('BUSINESS', 0.004127387814285715), ('PARENTING', 0.00403069818)]
insult
[('BLACK VOICES', 0.13720548), ('COMEDY', 0.11545324333333333), ('QUEER VOICES', 0.105889715), ('POLITICS', 0.09355055366568046), ('ENTERTAINMENT', 0.09273280733333333)]
profanity
[('QUEER VOICES', 0.051451176), ('BLACK VOICES', 0.03474322), ('PARENTING', 0.026260955800000003), ('HOME & LIVING', 0.021208711), ('ENTERTAINMENT', 0.020138974633333334)]
threat
[('BLACK VOICES', 0.09836589), ('THE WORLDPOST', 0.04817690166666667), ('SCIENCE', 0.023673484), ('QUEER VOICES', 0.022490766), ('PARENTING', 0.017929909)]
inflammatory
[('BLACK VOICES', 0.4420933), ('TRAVEL', 0.37276033), ('RELIGION', 0.35512886000000005), ('QUEER VOI

In [ ]:
print_top_means(gossipcop[gossipcop["label"]=="real"], feature_list)
print("DIVISAO")
print_top_means(gossipcop[gossipcop["label"]=="fake"], feature_list)

toxicity
[('CRIME', 0.16039132356), ('WOMEN', 0.15931562268627453), ('COMEDY', 0.1541376692577465), ('WEIRD NEWS', 0.1511314975846154), ('POLITICS', 0.1391911246517663)]
severe_toxicity
[('COMEDY', 0.022389951215211267), ('SCIENCE', 0.0222649691), ('WOMEN', 0.01717231080156863), ('PARENTS', 0.015169181326463416), ('PARENTING', 0.014919095543224162)]
insult
[('CRIME', 0.16124819265714288), ('COMEDY', 0.15468235566901412), ('WOMEN', 0.1486472801862745), ('POLITICS', 0.1458409485209239), ('WEIRD NEWS', 0.13939810164615385)]
profanity
[('SCIENCE', 0.09494350750909089), ('WOMEN', 0.08154851497058822), ('COMEDY', 0.0784525302084507), ('WEIRD NEWS', 0.06736772854153847), ('PARENTING', 0.06297076556260664)]
threat
[('CRIME', 0.07897368950285714), ('WELLNESS', 0.06251713741129034), ('GREEN', 0.057300909166666664), ('SCIENCE', 0.0435320415), ('POLITICS', 0.037272591875000004)]
inflammatory
[('WEIRD NEWS', 0.3352018064923077), ('WOMEN', 0.28456981426666667), ('SCIENCE', 0.275725105), ('STYLE', 0.

## Biggers difference between fake and real

In [ ]:
def biggest_differences_fake_real(df, feature):
  df = df.groupby(["news_category","label"])[feature].apply(list).apply(np.mean).reset_index()
  df = df.groupby("news_category")[feature].apply(list).apply(lambda x: x[1]-x[0] if len(x)==2 else 1000).sort_values().reset_index()
  return df


In [ ]:
aux = biggest_differences_fake_real(politifact, "toxicity")
aux[:10]

,news_category,toxicity
0,RELIGION,-0.389914
1,MEDIA,-0.143783
2,THE WORLDPOST,-0.116536
3,QUEER VOICES,-0.115295
4,PARENTING,-0.110418
5,GREEN,-0.104317
6,POLITICS,-0.101294
7,HEALTHY LIVING,-0.087016
8,COMEDY,-0.063866
9,HOME & LIVING,-0.053373


## Group By News Category

In [6]:
def groupby_news_category(df):
  df = df.groupby(['news_category']).size().reset_index(name='counts').sort_values(by="counts", ascending=False)
  return df

def topk_difference(df, topk=15):
  df_fake = groupby_news_category(df[df["label"]=="fake"])
  print(df_fake.head())

  df_real = groupby_news_category(df[df["label"]=="real"])
  print(df_real.head())

  print("Top-K", topk)
  print("Jaccard",jaccard_score(df_fake['news_category'].head(topk), df_real['news_category'].head(topk), average='micro'))
  print("Kendall", stats.kendalltau(df_fake['news_category'].head(topk), df_real['news_category'].head(topk)))

## News Categories
Most common news categories for each dataset

In [23]:
groupby_news_category(politifact)

,news_category,counts
14,POLITICS,576
12,NONE,142
1,BUSINESS,51
20,TECH,34
18,SPORTS,23
24,WELLNESS,20
5,ENTERTAINMENT,19
4,CRIME,15
21,THE WORLDPOST,11
13,PARENTING,11


In [30]:
values = groupby_news_category(politifact).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('POLITICS', 576), ('NONE', 142), ('BUSINESS', 51), ('TECH', 34), ('SPORTS', 23)]"

In [31]:
values = groupby_news_category(gossipcop).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('ENTERTAINMENT', 13873), ('POLITICS', 995), ('STYLE & BEAUTY', 980), ('QUEER VOICES', 577), ('SPORTS', 430)]"

In [36]:
values = groupby_news_category(kaggle).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('ENTERTAINMENT', 3352), ('POLITICS', 544), ('STYLE & BEAUTY', 192), ('QUEER VOICES', 151), ('SPORTS', 96)]"

In [37]:
values = groupby_news_category(liar).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('POLITICS', 9336), ('BUSINESS', 1616), ('WELLNESS', 252), ('THE WORLDPOST', 233), ('TRAVEL', 162)]"

In [38]:
values = groupby_news_category(coaid).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('POLITICS', 1830), ('WELLNESS', 1649), ('NONE', 500), ('IMPACT', 137), ('HEALTHY LIVING', 114)]"

In [39]:
values = groupby_news_category(celebrity).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('ENTERTAINMENT', 423), ('POLITICS', 27), ('STYLE & BEAUTY', 17), ('QUEER VOICES', 14), ('BLACK VOICES', 8)]"

In [40]:
values = groupby_news_category(fakenews).head().to_dict()
values = list(zip(list(values["news_category"].values()), list(values["counts"].values())))
str(sorted(values, key=lambda x: x[1], reverse=True))

"[('POLITICS', 193), ('SPORTS', 74), ('ENTERTAINMENT', 59), ('TECH', 43), ('BUSINESS', 34)]"

## Proportion of fake/real for each category

In [ ]:
def percentage_fake_real_category(df):
  results = []
  list_of_cats = list(df["news_category"].unique())
  for c in list_of_cats:
    aux = df[df["news_category"]==c]
    fake_count = len(aux[aux["label"]=="fake"])
    real_count = len(aux[aux["label"]=="real"])
    results.append((c, real_count, fake_count))
  return pd.DataFrame(results,columns=["news_category","n_real","n_fake"])


In [ ]:
percentage_fake_real_category(politifact)

,news_category,n_real,n_fake
0,BUSINESS,35,16
1,POLITICS,338,238
2,NONE,111,31
3,TECH,17,17
4,THE WORLDPOST,6,5
5,WELLNESS,14,6
6,HOME & LIVING,1,2
7,MEDIA,3,2
8,STYLE & BEAUTY,4,0
9,PARENTING,5,6


In [ ]:
percentage_fake_real_category(gossipcop)

,news_category,n_real,n_fake
0,ENTERTAINMENT,10209,3664
1,SPORTS,379,51
2,WOMEN,102,19
3,HOME & LIVING,98,12
4,BUSINESS,317,74
5,TECH,176,90
6,POLITICS,736,259
7,WELLNESS,186,28
8,QUEER VOICES,464,113
9,WEDDINGS,148,38


In [ ]:
percentage_fake_real_category(kaggle)

,news_category,n_real,n_fake
0,ENTERTAINMENT,1951,1401
1,BLACK VOICES,49,19
2,HOME & LIVING,21,4
3,POLITICS,270,274
4,MEDIA,8,15
5,STYLE & BEAUTY,143,49
6,TECH,37,31
7,PARENTING,31,13
8,PARENTS,22,7
9,WOMEN,13,8


In [ ]:
percentage_fake_real_category(liar)

,news_category,n_real,n_fake
0,POLITICS,1427,1871
1,SPORTS,30,23
2,BUSINESS,297,277
3,WELLNESS,39,50
4,RELIGION,2,17
5,BLACK VOICES,2,4
6,MEDIA,4,5
7,LATINO VOICES,3,1
8,COLLEGE,16,6
9,QUEER VOICES,19,30


In [ ]:
percentage_fake_real_category(coaid)

,news_category,n_real,n_fake
0,POLITICS,1738,92
1,WOMEN,11,1
2,SCIENCE,66,14
3,WELLNESS,1608,41
4,NONE,479,21
5,BUSINESS,107,7
6,MONEY,1,1
7,PARENTING,59,3
8,SPORTS,8,2
9,ENTERTAINMENT,12,4


## Mean for each category - Statistically Validated


In [ ]:
# feature_list = ['toxicity', 'severe_toxicity', 'insult', 'profanity',
#        'threat', 'inflammatory', 'identity_attack']
feature_list = ['insult','toxicity', 'inflammatory',]


def print_top_means(df, feature_list=feature_list, topk=5):
  for f in feature_list:
    # print("\n")
    cats = df["news_category"].unique()
    results_list = []
    lis = []

    for c in cats:
      aux = df[df["news_category"]==c]
      aux = aux[aux[f]>=0]
      fake_list = list(aux[aux["label"]=="fake"][f].values)
      real_list = list(aux[aux["label"]=="real"][f].values)

      if(len(fake_list) and len(real_list)):
        try:
          pvalue = stats.mannwhitneyu(fake_list, real_list)[1]
          if(pvalue<0.05):
            # print(c, np.round(pvalue,5), np.round(np.mean(fake_list) - np.mean(real_list),5))
            lis += [(c, np.round(np.mean(fake_list) - np.mean(real_list),4))]#f"({c}, {np.round(np.mean(fake_list) - np.mean(real_list),4)}) "
        except Exception as e:
          pass
    print(str(sorted(lis, key=lambda x: x[1])[-5:])[1:-1])


In [ ]:
print_top_means(politifact)

('TECH', -0.007), ('GREEN', 0.0764), ('POLITICS', 0.1239)
('TECH', 0.0049), ('WELLNESS', 0.0111), ('POLITICS', 0.0948), ('GREEN', 0.1043), ('PARENTING', 0.1104)
('TECH', -0.0508), ('PARENTING', 0.211), ('POLITICS', 0.2167), ('GREEN', 0.3235)


In [ ]:
print_top_means(gossipcop)

('ENTERTAINMENT', 0.0207), ('POLITICS', 0.0239), ('WEDDINGS', 0.0266), ('MEDIA', 0.0447), ('PARENTS', 0.0514)
('BUSINESS', -0.0048), ('ENTERTAINMENT', 0.0154), ('DIVORCE', 0.0185), ('WEDDINGS', 0.0218), ('PARENTS', 0.0389)
('HOME & LIVING', 0.0396), ('PARENTING', 0.047), ('ENTERTAINMENT', 0.0574), ('COMEDY', 0.1013), ('DIVORCE', 0.1097)


In [ ]:
print_top_means(kaggle)

('BUSINESS', -0.0037), ('ENTERTAINMENT', 0.0214), ('POLITICS', 0.0486), ('WEIRD NEWS', 0.0689), ('WOMEN', 0.1104)
('RELIGION', -0.0409), ('ENTERTAINMENT', 0.0252), ('POLITICS', 0.0309), ('WEIRD NEWS', 0.067)
('SPORTS', 0.0971), ('BLACK VOICES', 0.1044), ('POLITICS', 0.1358), ('CRIME', 0.1412), ('PARENTS', 0.1465)


In [ ]:
print_top_means(liar)

('THE WORLDPOST', 0.0072), ('BUSINESS', 0.0082), ('POLITICS', 0.0097), ('SPORTS', 0.0568), ('CRIME', 0.0728)
('POLITICS', 0.0112), ('THE WORLDPOST', 0.0299), ('SPORTS', 0.0601), ('ENTERTAINMENT', 0.0767), ('CRIME', 0.0911)
('WELLNESS', -0.0681), ('BUSINESS', 0.0156), ('POLITICS', 0.0159), ('THE WORLDPOST', 0.077)


In [ ]:
print_top_means(coaid)

('THE WORLDPOST', 0.0402), ('POLITICS', 0.0487), ('SCIENCE', 0.07), ('PARENTING', 0.1141), ('SPORTS', 0.2636)
('THE WORLDPOST', 0.0729), ('ENTERTAINMENT', 0.0749), ('SCIENCE', 0.0781), ('PARENTING', 0.1185), ('SPORTS', 0.2157)
('WELLNESS', 0.05), ('PARENTING', 0.1108), ('POLITICS', 0.1255), ('BUSINESS', 0.1893), ('SCIENCE', 0.204)


In [ ]:
print_top_means(celebrity)

('ENTERTAINMENT', 0.0616)
('ENTERTAINMENT', 0.0445)
('ENTERTAINMENT', 0.1175), ('POLITICS', 0.1648)


In [ ]:
print_top_means(fakenews)

('POLITICS', 0.0234), ('BUSINESS', 0.0358), ('ENTERTAINMENT', 0.0442), ('SPORTS', 0.0607)
('POLITICS', 0.0202)
('POLITICS', 0.1093)
